<a href="https://colab.research.google.com/github/susantaghosh1/nlp-notebooks/blob/develop/Text_classification_glue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
%%capture
!pip install datasets transformers[sentencepiece]
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install scipy sklearn

In [62]:
import torch

In [63]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [64]:
device

device(type='cuda')

In [65]:
!nvidia-smi

Tue May 17 16:12:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    30W /  70W |   7042MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [66]:
import transformers
print(transformers.__version__)

4.19.2


# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the 🤗 Transformers model to a text classification task of the GLUE Benchmark.

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

CoLA (Corpus of Linguistic Acceptability) : Determine if a sentence is grammatically correct or not.is a dataset containing sentences labeled grammatically correct or not.

MNLI (Multi-Genre Natural Language Inference) : Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)

MRPC (Microsoft Research Paraphrase Corpus) : Determine if two sentences are paraphrases from one another or not.

QNLI (Question-answering Natural Language Inference) : Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)

QQP (Quora Question Pairs2) : Determine if two questions are semantically equivalent or not.

RTE (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.

SST-2 (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.

STS-B (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.

WNLI (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the Trainer API to fine-tune a model on it. Each task is named by its acronym, with mnli-mm standing for the mismatched version of MNLI (so same training set as mnli but different validation and test sets):

In [67]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the Model Hub as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [68]:
task = "mrpc"
model_checkpoint = "bert-base-uncased"#"distilbert-base-uncased" ##choose distilbert to reduce training time. bert,roberta,deberta all the models based on transformer encoder architecture with classification head on top can be used
batch_size = 16

# Loading the dataset

We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions load_dataset and load_metric.

In [69]:
from datasets import load_dataset,load_metric

Apart from mnli-mm being a special code, we can directly pass our task name to those functions. load_dataset will cache the dataset to avoid downloading it again the next time you run this cell.

In [70]:
actual_task = "mnli" if task == "mnli-mm" else task
print(f"going to load {actual_task} dataset as well as metric for evaluation")
dataset = load_dataset('glue',actual_task)
metric = load_metric('glue',actual_task)

going to load mrpc dataset as well as metric for evaluation


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [71]:
print(f"loaded dataset : {dataset}")

loaded dataset : DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


let's explore the dataset

In [74]:
dataset['train'].features #features of the dataset

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

In [75]:
dataset['train'][0] # first element of the dataset

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

In [76]:
dataset['train'][:5]

{'idx': [0, 1, 2, 3, 4],
 'label': [1, 0, 1, 0, 1],
 'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
  'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
  'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .'],
 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .",
  'Tab shares jumped 20

In [77]:
print(dataset['train'][:5].keys())
for k,v in dataset['train'][:5].items():
    print(f"shape of {k} : {len(v)}")

dict_keys(['sentence1', 'sentence2', 'label', 'idx'])
shape of sentence1 : 5
shape of sentence2 : 5
shape of label : 5
shape of idx : 5


In [80]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

You can call its compute method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [81]:
import numpy as np
fake_preds = np.random.randint(0, 2, size=(64,2)) # as in MNLI dataset we have 3 labels, so here I am generating 64 numbers between 0 and 3
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=np.argmax(fake_preds,axis=-1),references=fake_labels)

{'accuracy': 0.625, 'f1': 0.5384615384615384}

Note that load_metric has loaded the proper metric associated to your task, which is:

for CoLA: Matthews Correlation Coefficient

for MNLI (matched or mismatched): Accuracy
for MRPC: Accuracy and F1 score
for QNLI: Accuracy
for QQP: Accuracy and F1 score
for RTE: Accuracy
for SST-2: Accuracy
for STS-B: Pearson Correlation Coefficient and Spearman's_Rank_Correlation_Coefficient
for WNLI: Accuracy
so the metric object only computes the one(s) needed for your task.

# Tokenizing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

Pls note : if you have domain specific corpus or your fine-tuning data varies too much with respect to pre-training data of the model [like BERT was pretrained with wikipedia and book corpus with masked language modelling [MLM] and next sentence prediction [NSP] task] then it's recommended to first train your tokenizer [Bert-like [wordpiece tokenizer which bert uses]] on your domain data and then do a re-training of BERT with the trained tokenizer on Masked language Modelling [MLM] task. This is called domain adoptaion of a pre-trained language model. Then you finetune that domain adopted model in any of the downstream tasks like BERT can be fine-tuned to sequence or document classification [binary/multi-class/multi-label], token classification or Named Entity Recognition [NER] , Extractive Question Answering [ Single Span], multiple Choice Question  Answering .

In [82]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

We pass along use_fast=True to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [83]:
encoding = tokenizer("Hello, this one sentence!", "And this sentence goes with it.") #tokenizer can accept pairs of sentences [ use for tasks where sentence pairs are required like MNLI,STSB etc] ]

In [84]:
type(encoding)

transformers.tokenization_utils_base.BatchEncoding

In [85]:
encoding #encoding contains all the necessary info to pass throught the transformer model

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [86]:
encoding = tokenizer("Hello, this one sentence!") #single sentence [use for CoLa task,sentiment analysis,imdb movie review,sst2
                                                                #document classification tasks]
encoding

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [87]:
encoding = tokenizer(["Hello, this one sentence!","This is second sentence!!"]) #batching of single sentence.. tokenizer can accept list of sentences
encoding

{'input_ids': [[101, 7592, 1010, 2023, 2028, 6251, 999, 102], [101, 2023, 2003, 2117, 6251, 999, 999, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [88]:
encoding = tokenizer(["Hello, this is 1st sentence!","This is 2nd sentence!!"
                     ,"This is 3rd sentence","this is 4th sentence"],
                     ["Hello, this is the paraphrase of 1st sentence!",
                      "This is the paraphrase of 2nd sentence!!",
                      "I contradict 3rd setence!","I am also not a paraphrase of 4th sentence!"],)
# we will use this to encode all our training data in batches to save time
encoding

{'input_ids': [[101, 7592, 1010, 2023, 2003, 3083, 6251, 999, 102, 7592, 1010, 2023, 2003, 1996, 11498, 8458, 23797, 1997, 3083, 6251, 999, 102], [101, 2023, 2003, 3416, 6251, 999, 999, 102, 2023, 2003, 1996, 11498, 8458, 23797, 1997, 3416, 6251, 999, 999, 102], [101, 2023, 2003, 3822, 6251, 102, 1045, 24528, 29201, 3822, 2275, 10127, 999, 102], [101, 2023, 2003, 4343, 6251, 102, 1045, 2572, 2036, 2025, 1037, 11498, 8458, 23797, 1997, 4343, 6251, 999, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [89]:
tokenizer.decode(encoding['input_ids'][0],skip_special_tokens=False) #decoding

'[CLS] hello, this is 1st sentence! [SEP] hello, this is the paraphrase of 1st sentence! [SEP]'

In [90]:
for idx,input_id in enumerate(encoding['input_ids']):
    print(tokenizer.decode(input_id,skip_special_tokens=False))

[CLS] hello, this is 1st sentence! [SEP] hello, this is the paraphrase of 1st sentence! [SEP]
[CLS] this is 2nd sentence!! [SEP] this is the paraphrase of 2nd sentence!! [SEP]
[CLS] this is 3rd sentence [SEP] i contradict 3rd setence! [SEP]
[CLS] this is 4th sentence [SEP] i am also not a paraphrase of 4th sentence! [SEP]


In [91]:
#names of the columns containing the sentence(s)
task_types={
     "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [92]:
sentence1_key, sentence2_key = task_types[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
Sentence 2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .


In [93]:
#let's tokenize our train,test and validation data
def tokenize_data(sample):
    if sentence2_key is None:
        return tokenizer(sample[sentence1_key],truncation=True,padding='max_length',max_length=256)
    else:
        return tokenizer(sample[sentence1_key],sample[sentence2_key],truncation=True,padding='max_length',max_length=256)

The above function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [94]:
encoding=tokenize_data(dataset['train'][0])
encoding

{'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [95]:
tokenizer.decode(encoding['input_ids'],skip_special_tokens=False) #decoding

'[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] referring to him as only " the witness ", amrozi accused his brother of deliberately distorting his evidence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [96]:
encoding=tokenize_data(dataset['train'][:5])
encoding

{'input_ids': [[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077

In [97]:
for k,v in encoding.items():
    print(f"size of {k} : {len(v)}")  # each of key in dict is a list of lists

size of input_ids : 5
size of token_type_ids : 5
size of attention_mask : 5


In [98]:
for idx,input_id in enumerate(encoding['input_ids']):
    print(tokenizer.decode(input_id,skip_special_tokens=False))

[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] referring to him as only " the witness ", amrozi accused his brother of deliberately distorting his evidence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [99]:
%%time
encoded_dataset = dataset.map(tokenize_data,batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

CPU times: user 2.01 s, sys: 49.4 ms, total: 2.06 s
Wall time: 1.53 s


In [100]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

 the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

# Fine-tuning the model

Now we will train the model. Now  we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the AutoModelForSequenceClassification class. Like with the tokenizer, the from_pretrained method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels).

What actually we are doing is taking the vanilla BERT and then adding a liner layer on top of it based on our downstream task and then fine tuning only the linear layer by transferring the knwledge from bert.

In [101]:
from transformers import AutoModelForSequenceClassification,Trainer,TrainingArguments
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/re

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

We can train the model with hf Trainer, native pytorch training loop , hf accelerate  and with tensorflow. Here I will prefer to sue Trainer.

To instantiate a Trainer, we will need to define two more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [102]:
TrainingArguments??

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the Trainer to load the best model it saved (according to metric_name) at the end of training.

In [103]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(f"{model_name}-finetuned-{task}",weight_decay=0.01,
                                  evaluation_strategy = "epoch",save_strategy = "epoch",
                                  learning_rate=2e-5,per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,num_train_epochs=5,
                                  load_best_model_at_end=True,metric_for_best_model=metric_name,)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


The last thing to define for our Trainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [106]:
def compute_metrics(eval_preds):
    logits,labels = eval_preds
    if task != "stsb":
        predictions = np.argmax(logits, axis=-1)
    else:
        predictions = logits[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [107]:
encoded_dataset = encoded_dataset.remove_columns(column_names=['sentence1', 'sentence2','idx']) #remove the columns which are not required by the model
encoded_dataset = encoded_dataset.rename_columns(column_mapping={'label':'labels'}) #renaming label column to labels as all transformers model needs this
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [108]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [109]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1150


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.388655,0.821078,0.875639
2,No log,0.440001,0.857843,0.903654
3,0.413000,0.513491,0.850490,0.895369
4,0.413000,0.641961,0.855392,0.899489
5,0.141000,0.635018,0.865196,0.904348


***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-mrpc/checkpoint-230
Configuration saved in bert-base-uncased-finetuned-mrpc/checkpoint-230/config.json
Model weights saved in bert-base-uncased-finetuned-mrpc/checkpoint-230/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-mrpc/checkpoint-230/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-mrpc/checkpoint-230/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-mrpc/checkpoint-460
Configuration saved in bert-base-uncased-finetuned-mrpc/checkpoint-460/config.json
Model weights saved in bert-base-uncased-finetuned-mrpc/checkpoint-460/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-mrpc/checkpoint-460/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-mrpc/c

TrainOutput(global_step=1150, training_loss=0.25037255411562714, metrics={'train_runtime': 885.6637, 'train_samples_per_second': 20.708, 'train_steps_per_second': 1.298, 'total_flos': 2412728377651200.0, 'train_loss': 0.25037255411562714, 'epoch': 5.0})

In [110]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 408
  Batch size = 16


{'epoch': 5.0,
 'eval_accuracy': 0.8651960784313726,
 'eval_f1': 0.9043478260869565,
 'eval_loss': 0.6350182890892029,
 'eval_runtime': 6.1758,
 'eval_samples_per_second': 66.064,
 'eval_steps_per_second': 4.21}